<a href="https://colab.research.google.com/github/james-lucius/qosf_cohort11_task3/blob/main/qosf_cohort11_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00


In [5]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import random

## 1. Noise Model Function
*   Created a function that adds Pauli X errors with probability a and Pauli Z errors with probability b.
*    The function properly returns a modified circuit with noise applied

In [6]:
# Task 1: Simple Noise Model Function
def noise_model(a, b, circuit):
    """
    Add random Pauli noise to a quantum circuit.

    Parameters:
    a: probability of Pauli X error
    b: probability of Pauli Z error
    circuit: input quantum circuit

    Returns:
    circuit_with_noise: modified circuit with noise added
    """
    # Create a copy of the circuit to avoid modifying the original
    circuit_with_noise = circuit.copy()

    # Get the number of qubits
    num_qubits = circuit.num_qubits

    # For each qubit, randomly apply Pauli errors
    for qubit in range(num_qubits):
        rand = random.random()

        if rand < a:
            # Apply Pauli X with probability 'a'
            circuit_with_noise.x(qubit)
        elif rand < a + b:
            # Apply Pauli Z with probability 'b'
            circuit_with_noise.z(qubit)
        # With probability (1 - a - b), no error is applied

    return circuit_with_noise


In [8]:
# Test the noise model
def test_noise_model():
    # Create a simple test circuit
    qc = QuantumCircuit(3, 3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)

    print("Original circuit:")
    print(qc)

    # Apply noise model with a=0.1, b=0.1
    noisy_circuit = noise_model(0.1, 0.1, qc)
    print("\nCircuit with noise applied:")
    print(noisy_circuit)

    return noisy_circuit

## 2. Quantum Repetition Code
*   Implements the 3-qubit repetition code.
*   Important finding: The repetition code ONLY works for X (bit-flip) errors, NOT for Z (phase-flip) errors.

*   This is why the method doesn't work for Z errors; the repetition code's syndrome extraction cannot detect phase errors.

In [9]:
# Task 2: Quantum Repetition Code
def quantum_repetition_code_with_noise(a, b):
    """
    Implement quantum repetition code with noise model.
    This encodes a single logical qubit into 3 physical qubits.
    """
    # Create circuit with 3 qubits for encoding + 2 ancilla for syndrome measurement
    qr = QuantumRegister(5, 'q')
    cr = ClassicalRegister(5, 'c')
    qc = QuantumCircuit(qr, cr)

    # Prepare initial state |0⟩

    # Encoding: |0⟩ -> |000⟩, |1⟩ -> |111⟩
    qc.cx(qr[0], qr[1])
    qc.cx(qr[0], qr[2])

    qc.barrier()

    # Apply noise ONLY to X errors (for repetition code to work)
    # Z errors cannot be detected by this code
    for qubit in range(3):
        if random.random() < a:
            qc.x(qr[qubit])

    qc.barrier()

    # Syndrome extraction
    # Ancilla qr[3] checks parity of qubits 0 and 1
    qc.cx(qr[0], qr[3])
    qc.cx(qr[1], qr[3])

    # Ancilla qr[4] checks parity of qubits 1 and 2
    qc.cx(qr[1], qr[4])
    qc.cx(qr[2], qr[4])

    # Measure syndrome qubits
    qc.measure(qr[3], cr[3])
    qc.measure(qr[4], cr[4])

    # Error correction based on syndrome
    # This would be done classically based on measurement results
    # Syndrome 00: no error
    # Syndrome 11: error on qubit 1
    # Syndrome 10: error on qubit 0
    # Syndrome 01: error on qubit 2

    # Measure data qubits
    qc.measure(qr[0], cr[0])
    qc.measure(qr[1], cr[1])
    qc.measure(qr[2], cr[2])

    return qc

## 3. Shor Code
*   Implements the 9-qubit Shor code.
*   Uses concatenation of bit-flip and phase-flip codes.
*   Can correct both X and Z errors (any single-qubit error)

In [10]:
# Task 3: Shor Code
def shor_code_with_noise(a, b):
    """
    Implement 9-qubit Shor code with noise model.
    Protects against single qubit errors (both X and Z).
    """
    # Create circuit with 9 data qubits + ancilla for syndrome
    qr = QuantumRegister(11, 'q')  # 9 data + 2 ancilla
    cr = ClassicalRegister(11, 'c')
    qc = QuantumCircuit(qr, cr)

    # Encode |0⟩ into logical |0⟩_L
    # First level: bit-flip encoding
    qc.cx(qr[0], qr[3])
    qc.cx(qr[0], qr[6])

    # Second level: phase-flip encoding (apply H gates and CNOT)
    for i in [0, 3, 6]:
        qc.h(qr[i])
        qc.cx(qr[i], qr[i+1])
        qc.cx(qr[i], qr[i+2])

    qc.barrier()

    # Apply noise model to all 9 qubits
    for qubit in range(9):
        rand = random.random()
        if rand < a:
            qc.x(qr[qubit])
        elif rand < a + b:
            qc.z(qr[qubit])

    qc.barrier()

    # Syndrome measurement for X errors (bit-flip)
    # Within each block of 3
    for block_start in [0, 3, 6]:
        # Use ancilla to check parity
        qc.cx(qr[block_start], qr[9])
        qc.cx(qr[block_start+1], qr[9])
        qc.cx(qr[block_start+1], qr[10])
        qc.cx(qr[block_start+2], qr[10])
        # In practice, measure and correct here

    # Syndrome measurement for Z errors (phase-flip)
    # Between blocks (after applying H gates to go to X basis)
    for i in [0, 3, 6]:
        qc.h(qr[i])
        qc.h(qr[i+1])
        qc.h(qr[i+2])

    # Measure all qubits for demonstration
    for i in range(11):
        qc.measure(qr[i], cr[i])

    return qc

## 4. Hamming [7,4,3] Code
*   Encodes 4 logical qubits into 7 physical qubits
*   Implements parity checks for syndrome extraction
*   More efficient than Shor code but limited to bit-flip corrections

In [11]:
# Task 4: Hamming [7,4,3] Code
def hamming_code_with_noise(a, b):
    """
    Implement Hamming [7,4,3] code with noise model.
    Encodes 4 logical qubits into 7 physical qubits.
    Can detect and correct single-bit errors.
    """
    # Create circuit with 7 qubits + 3 syndrome qubits
    qr = QuantumRegister(10, 'q')
    cr = ClassicalRegister(10, 'c')
    qc = QuantumCircuit(qr, cr)

    # Initialize 4 data qubits (can prepare any state)
    # For demonstration, prepare a simple state
    # qc.h(qr[0])  # Data qubit 1
    # qc.x(qr[1])  # Data qubit 2

    # Hamming encoding
    # Data qubits: q[0], q[1], q[2], q[3]
    # Parity qubits: q[4], q[5], q[6]

    # Calculate parity bits
    # P1 (q[4]) = D1 ⊕ D2 ⊕ D4
    qc.cx(qr[0], qr[4])
    qc.cx(qr[1], qr[4])
    qc.cx(qr[3], qr[4])

    # P2 (q[5]) = D1 ⊕ D3 ⊕ D4
    qc.cx(qr[0], qr[5])
    qc.cx(qr[2], qr[5])
    qc.cx(qr[3], qr[5])

    # P3 (q[6]) = D2 ⊕ D3 ⊕ D4
    qc.cx(qr[1], qr[6])
    qc.cx(qr[2], qr[6])
    qc.cx(qr[3], qr[6])

    qc.barrier()

    # Apply noise model to all 7 qubits
    for qubit in range(7):
        rand = random.random()
        if rand < a:
            qc.x(qr[qubit])
        elif rand < a + b:
            qc.z(qr[qubit])

    qc.barrier()

    # Syndrome extraction using ancilla qubits 7, 8, 9
    # S1 checks: positions 1, 3, 5, 7
    qc.cx(qr[0], qr[7])  # pos 1
    qc.cx(qr[2], qr[7])  # pos 3
    qc.cx(qr[4], qr[7])  # pos 5
    qc.cx(qr[6], qr[7])  # pos 7

    # S2 checks: positions 2, 3, 6, 7
    qc.cx(qr[1], qr[8])  # pos 2
    qc.cx(qr[2], qr[8])  # pos 3
    qc.cx(qr[5], qr[8])  # pos 6
    qc.cx(qr[6], qr[8])  # pos 7

    # S3 checks: positions 4, 5, 6, 7
    qc.cx(qr[3], qr[9])  # pos 4
    qc.cx(qr[4], qr[9])  # pos 5
    qc.cx(qr[5], qr[9])  # pos 6
    qc.cx(qr[6], qr[9])  # pos 7

    # Measure syndrome and data qubits
    for i in range(10):
        qc.measure(qr[i], cr[i])

    return qc

## 5. Key Differences:

Shor: 1 logical → 9 physical qubits, protects against all single-qubit errors

Hamming: 4 logical → 7 physical qubits, mainly for bit-flip errors

     

Shor: Higher overhead but complete protection

Hamming: Better encoding rate but limited protection

In [12]:
# Task 5: Differences between Shor and Hamming codes
def compare_codes():
    """
    Compare and contrast Shor and Hamming codes.
    """
    differences = """
    Differences between Shor and Hamming codes:

    1. **Purpose and Protection**:
       - Shor Code: Protects against both bit-flip (X) and phase-flip (Z) errors
       - Hamming Code: Originally designed for classical error correction,
         adapted for quantum to protect primarily against bit-flip errors

    2. **Encoding Rate**:
       - Shor Code: Encodes 1 logical qubit into 9 physical qubits (rate = 1/9)
       - Hamming [7,4,3]: Encodes 4 logical qubits into 7 physical qubits (rate = 4/7)

    3. **Error Correction Capability**:
       - Shor Code: Can correct any single-qubit error (X, Y, or Z)
       - Hamming Code: Can detect and correct single bit-flip errors,
         but not phase errors without modification

    4. **Code Structure**:
       - Shor Code: Concatenated code combining bit-flip and phase-flip codes
       - Hamming Code: Linear code based on parity check matrix

    5. **Overhead**:
       - Shor Code: Higher overhead (9 qubits for 1 logical qubit)
       - Hamming Code: Lower overhead (7 qubits for 4 logical qubits)

    6. **Quantum vs Classical Origin**:
       - Shor Code: Specifically designed for quantum error correction
       - Hamming Code: Classical code adapted for quantum use
    """

    return differences

## 6. Major Challenges Identified:

*   No-cloning theorem prevents simple copying
*   Measurement collapses superposition
*   Continuous errors must be discretized
*   High overhead requirements
*   Fault-tolerance requirements

In [13]:
# Task 6: Challenges in building error-correcting codes
def challenges_analysis():
    """
    Analyze challenges encountered in building error-correcting codes.
    """
    challenges = """
    Challenges detected in building error-correcting codes:

    1. **No-Cloning Theorem**:
       - Cannot simply copy quantum states for redundancy
       - Must use entanglement to spread information across multiple qubits

    2. **Measurement Destroys Superposition**:
       - Cannot directly measure data qubits to check for errors
       - Must use ancilla qubits and syndrome extraction

    3. **Continuous Error Model**:
       - Quantum errors are continuous (rotation errors)
       - Must discretize to Pauli errors for correction

    4. **Decoherence During Correction**:
       - Error correction itself takes time during which more errors occur
       - Need fast, fault-tolerant syndrome extraction

    5. **Limited Correction Capability**:
       - Repetition code cannot detect Z errors
       - Need more sophisticated codes for complete protection

    6. **Scalability**:
       - Large overhead (9 qubits for 1 logical qubit in Shor code)
       - Need better codes with higher rates for practical quantum computing

    7. **Fault-Tolerant Implementation**:
       - Syndrome extraction circuits can propagate errors
       - Need careful circuit design to prevent error spread

    8. **Threshold Requirements**:
       - Physical error rate must be below threshold for QEC to help
       - Current hardware often near or above threshold
    """

    return challenges

In [19]:
# Main execution and testing
if __name__ == "__main__":
    print("Task 3: Quantum Error Correction\n")
    print("="*50)

    # Test Task 1
    print("\nTask 1: Testing Noise Model")
    print("-"*30)
    test_noise_model()

    # Test Task 2
    print("\n\nTask 2: Quantum Repetition Code")
    print("-"*30)
    rep_circuit = quantum_repetition_code_with_noise(0.1, 0.0)
    print("Note: Repetition code only works for X errors, not Z errors!")
    print("Circuit created with", rep_circuit.num_qubits, "qubits")

    # Test Task 3
    print("\n\nTask 3: Shor Code")
    print("-"*30)
    shor_circuit = shor_code_with_noise(0.1, 0.1)
    print("Shor code circuit created with", shor_circuit.num_qubits, "qubits")

    # Test Task 4
    print("\n\nTask 4: Hamming Code")
    print("-"*30)
    hamming_circuit = hamming_code_with_noise(0.1, 0.1)
    print("Hamming code circuit created with", hamming_circuit.num_qubits, "qubits")

    # Task 5
    print("\n\nTask 5: Code Comparison")
    print("-"*30)
    print(compare_codes())

    # Task 6
    print("\n\nTask 6: Challenges Analysis")
    print("-"*30)
    print(challenges_analysis())

Task 3: Quantum Error Correction


Task 1: Testing Noise Model
------------------------------
Original circuit:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐     
q_1: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘
c: 3/═══════════════
                    

Circuit with noise applied:
     ┌───┐               
q_0: ┤ H ├──■────────────
     └───┘┌─┴─┐          
q_1: ─────┤ X ├──■───────
          └───┘┌─┴─┐┌───┐
q_2: ──────────┤ X ├┤ X ├
               └───┘└───┘
c: 3/════════════════════
                         


Task 2: Quantum Repetition Code
------------------------------
Note: Repetition code only works for X errors, not Z errors!
Circuit created with 5 qubits


Task 3: Shor Code
------------------------------
Shor code circuit created with 11 qubits


Task 4: Hamming Code
------------------------------
Hamming code circuit created with 10 qubits


Task 5: Code Comparison
------------------------------

    Differences between Shor and